In [8]:
import scipy.io as sio
import numpy as np
import pandas as pd
import h5py
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import v2v as v2v
from Colormaps import *

%matplotlib inline

# Load Data

In [9]:
data_folder = '/home/maggiemae/Desktop/Current_Data/vim1_redo/'

area_list= ['v1', 'v2', 'v3', 'v4', 'v3ab', 'LO', 'wm', 'air']
roiNum = {'v1':1,'v2':2,'v3':3,'v4':6,'v3ab':4,'LO':7, 'wm':10 ,'air':20}
lambda_list = np.logspace(1,6,num=6)

In [45]:
# LOAD Data. Note S1 Repeated Trial Analysis has white matter/ noise betas included

fn = data_folder + 'betas/Rank1GLM_Beta_Responses.mat'

S1VoxelNanMaskRep = sio.loadmat(fn)['voxelNanMaskS1N'][0,:].astype('bool')
S1VoxelNanMaskSing = sio.loadmat(fn)['voxelNanMaskS1'][0,:].astype('bool')

S1TrainDataRep = sio.loadmat(fn)['dataTrnS1'][:,S1VoxelNanMaskRep]
S1TestDataRep  = sio.loadmat(fn)['dataValS1'][:,S1VoxelNanMaskRep]

S1TrainDataSing = sio.loadmat(fn)['dataTrnSingleS1'][:,S1VoxelNanMaskSing]
S1TestDataSing  = sio.loadmat(fn)['dataValSingleS1'][:,S1VoxelNanMaskSing]

S1RoiMaskRep = sio.loadmat(fn)['roiS1N'][0,S1VoxelNanMaskRep]
S1RoiMaskSing= sio.loadmat(fn)['roiS1'][0,S1VoxelNanMaskSing]

# ---------- #

S2VoxelNanMask= sio.loadmat(fn)['voxelNanMaskS2'][0,:].astype('bool')

S2TrainDataRep = sio.loadmat(fn)['dataTrnS2'][:,S2VoxelNanMask]
S2TestDataRep  = sio.loadmat(fn)['dataValS2'][:,S2VoxelNanMask]

S2TrainDataSing = sio.loadmat(fn)['dataTrnSingleS2'][:,S2VoxelNanMask]
S2TestDataSing  = sio.loadmat(fn)['dataValSingleS2'][:,S2VoxelNanMask]

S2RoiMask = sio.loadmat(fn)['roiS2'][0,S2VoxelNanMask]

# Combine V3a & V3b and Both White Matter ROIs for S1

S1RoiMaskRep[S1RoiMaskRep==5]=4
S1RoiMaskSing[S1RoiMaskSing==5]=4

S1RoiMaskRep[S1RoiMaskRep==12]=10
S1RoiMaskSing[S1RoiMaskSing==12]=10

S2RoiMask[S2RoiMask==5]=4
S2RoiMask[S2RoiMask==12]=10

if any([len(S1TrainDataRep)!=1750, len(S1TestDataRep)!=120,
        len(S1TrainDataSing)!=3500, len(S1TestDataSing)!= 1560,
        len(S2TrainDataRep)!=1750, len(S2TestDataRep)!=120,
        len(S2TrainDataSing)!=3500, len(S2TestDataSing)!= 1560]):
    print 'ERROR INCORRECT TRIAL NUMBERS FOR VIM1'

In [46]:
print 'Subject 1 has %d total voxels. V1: %d, V2: %d, V3: %d, V4: %d, V3ab: %d, LO: %d, WM: %d, Air: %d'%(\
        len(S1RoiMaskRep), (S1RoiMaskRep==1).sum(), (S1RoiMaskRep==2).sum(), (S1RoiMaskRep==3).sum(),
        (S1RoiMaskRep==6).sum(), (S1RoiMaskRep==4).sum(), (S1RoiMaskRep==7).sum(), (S1RoiMaskRep==10).sum(),
        (S1RoiMaskRep==20).sum())

print 'Subject 2 has %d total voxels. V1: %d, V2: %d, V3: %d, V4: %d, V3ab: %d, LO: %d'%(\
        len(S2RoiMask), (S2RoiMask==1).sum(), (S2RoiMask==2).sum(), (S2RoiMask==3).sum(),
        (S2RoiMask==6).sum(), (S2RoiMask==4).sum(), (S2RoiMask==7).sum())


Subject 1 has 24353 total voxels. V1: 1294, V2: 2095, V3: 1814, V4: 1541, V3ab: 798, LO: 928, WM: 541, Air: 794
Subject 2 has 21832 total voxels. V1: 1433, V2: 1890, V3: 1775, V4: 1022, V3ab: 1048, LO: 358


### Repeated Trial Models

In [ ]:
# Keep at least WTs dataframe named seperately, will need later on for adjustments
df_cc_S1_avg, df_wts_S1_avg, best_lambs_S1_avg = v2v.vox2vox(area_list, area_list, S1TrainDataRep, 
                                                                      S1TestDataRep, lambda_list, 'average', 
                                                                      .2, roiNum, S1RoiMaskRep)

df_cc_S1_avg.to_pickle(data_folder + 'results/voxel2voxel/S1_avg_cc.pkl') 
df_wts_S1_avg.to_pickle(data_folder + 'results/voxel2voxel/S1_avg_wts.pkl')  
best_lambs_S1_avg.to_pickle(data_folder + 'results/voxel2voxel/S1_avg_lambdas.pkl') 

In [48]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6], area_list[:6], S2TrainDataRep, S2TestDataRep, lambda_list, 
                                        'average', .2, roiNum, S2RoiMask)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S2_avg_cc.pkl') 
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S2_avg_wts.pkl')  
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S2_avg_lambdas.pkl') 

### Matched Trial Models

In [ ]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6],area_list[:6],S1TrainDataSing,S1TestDataSing, lambda_list, 
                                        'average', .2, roiNum, S1RoiMaskSing)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S1_single_match_cc.pkl') 
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S1_single_match_wts.pkl')  
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S1_single_match_lambdas.pkl') 

In [49]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6],area_list[:6],S2TrainDataSing,S2TestDataSing, lambda_list, 
                                        'average',.2, roiNum, S2RoiMask)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S2_single_match_cc.pkl') 
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S2_single_match_wts.pkl')  
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S2_single_match_lambdas.pkl') 

### Mixed Trial Models

#### The next cell reformats the single trial data so that it can easily be used by the source code to match the correct trials with each other

In [97]:
trainTrialOrder = sio.loadmat(fn)['trnTrialOrderSingle'][0,:]
tstTrialOrder = sio.loadmat(fn)['valTrialOrderSingle'][0,:]

first_instance = [trainTrialOrder.tolist().index(i) for i in range(1750)]
second_instance = np.ones(3500, dtype='bool')
second_instance[first_instance]=0

tstOrder = np.argsort(tstTrialOrder, kind='stable')

S1TrainDataSingMix = np.zeros_like(S1TrainDataSing)
S1TrainDataSingMix[:1750] = S1TrainDataSing[first_instance]
S1TrainDataSingMix[1750:] = S1TrainDataSing[second_instance]

S2TrainDataSingMix = np.zeros_like(S2TrainDataSing)
S2TrainDataSingMix[:1750] = S2TrainDataSing[first_instance]
S2TrainDataSingMix[1750:] = S2TrainDataSing[second_instance]

S1TestDataSingMix = S1TrainDataSing[np.argsort(tstTrialOrder, kind='stable')]
S2TestDataSingMix = S2TrainDataSing[np.argsort(tstTrialOrder, kind='stable')]


In [98]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6], area_list[:6], S1TrainDataSingMix, S1TestDataSingMix, 
                                   lambda_list, 'switch', .2, roiNum, S1RoiMaskSing)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S1_mix_cc.pkl') 
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S1_mix_wts.pkl')  
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S1_mix_lambdas.pkl') 

In [205]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6], area_list[:6], S2TrainDataSingMix, S2TestDataSingMix, 
                                   lambda_list, 'switch', .2, roiNum, S2RoiMask)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S2_mix_cc.pkl') 
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S2_mix_wts.pkl')  
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S2_mix_lambdas.pkl') 

### Cross Models

In [99]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6], area_list[:6], S1TrainDataRep, S1TestDataRep, lambda_list, 
                                        'cross',.2, roiNum, S1RoiMaskRep, S2TrainDataRep, S2TestDataRep, S2RoiMask)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S1_src_cross_cc.pkl')
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S1_src_cross_wts.pkl')
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S1_src_cross_lambdas.pkl')

In [37]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6], area_list[:6], S2TrainDataRep, S2TestDataRep, lambda_list, 
                                        'cross',.2, roiNum, S2RoiMask, S1TrainDataRep,S1TestDataRep, S1RoiMaskRep)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S2_src_cross_cc.pkl') 
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S2_src_cross_wts.pkl')  
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S2_src_cross_lambdas.pkl') 

### Adjusted Models

In [108]:
if 'df_wts_S1_avg' not in locals():
    
    print 'Please Load S1 Repeated Trial Vox2Vox Weight Data Frame as df_wts_S1_avg'

In [ ]:
origTrn = np.concatenate([S1TrainDataRep[:,S1RoiMaskRep==1],S1TrainDataRep[:,S1RoiMaskRep==2],
                          S1TrainDataRep[:,S1RoiMaskRep==3],S1TrainDataRep[:,S1RoiMaskRep==6],
                          S1TrainDataRep[:,S1RoiMaskRep==4],S1TrainDataRep[:,S1RoiMaskRep==7]],axis=1)
origTst = np.concatenate([S1TestDataRep[:,S1RoiMaskRep==1],S1TestDataRep[:,S1RoiMaskRep==2],
                          S1TestDataRep[:,S1RoiMaskRep==3],S1TestDataRep[:,S1RoiMaskRep==6],
                          S1TestDataRep[:,S1RoiMaskRep==4],S1TestDataRep[:,S1RoiMaskRep==7]],axis=1)

In [124]:
src = 'wm'

wts = np.concatenate(df_wts_S1_avg.loc[src][:6])
predTrnWM = np.dot(S1TrainDataRep[:,S1RoiMaskRep==roiNum[src]], wts.T)
predTstWM = np.dot(S1TestDataRep[:,S1RoiMaskRep==roiNum[src]], wts.T)

trnAdj = origTrn - predTrnWM
tstAdj = origTst - predTstWM

roiAdj = np.concatenate([np.repeat(1, (S1RoiMaskRep==1).sum()),np.repeat(2, (S1RoiMaskRep==2).sum()),
                         np.repeat(3, (S1RoiMaskRep==3).sum()),np.repeat(6, (S1RoiMaskRep==6).sum()),
                         np.repeat(4, (S1RoiMaskRep==4).sum()),np.repeat(7, (S1RoiMaskRep==7).sum())])

In [109]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6], area_list[:6], trnAdj, tstAdj, lambda_list, 'average', .2, 
                                        roiNum, roiAdj)

df_cc_S1_avg.to_pickle(data_folder + 'results/voxel2voxel/S1_wm_adj_cc.pkl') 
df_wts_S1_avg.to_pickle(data_folder + 'results/voxel2voxel/S1_wm_adj_wts.pkl')  
best_lambs_S1_avg.to_pickle(data_folder + 'results/voxel2voxel/S1_wm_adj_lambdas.pkl') 

In [126]:
src = 'air'

wts = np.concatenate(df_wts_S1_avg.loc[src][:6])
predTrnAir = np.dot(S1TrainDataRep[:,S1RoiMaskRep==roiNum[src]], wts.T)
predTstAir = np.dot(S1TestDataRep[:,S1RoiMaskRep==roiNum[src]], wts.T)

trnAdj = origTrn - predTrnAir
tstAdj = origTst - predTstAir

In [110]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6], area_list[:6], trnAdj, tstAdj, lambda_list, 'average',.2, 
                                        roiNum, roiAdj)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S1_air_adj_cc.pkl') 
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S1_air_adj_wts.pkl')  
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S1_air_adj_lambdas.pkl') 

In [128]:
trnAdj = origTrn - predTrnAir - predTrnWm
tstAdj = origTst - predTstAir - predTrnWm

df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6], area_list[:6], trnAdj, tstAdj, lambda_list, 'average', .2, 
                                        roiNum, roiAdj)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S1_both_adj_cc.pkl') 
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S1_both_adj_wts.pkl')  
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S1_both_adj_lambdas.pkl') 

In [157]:
src = 'v1'
sn = 1

predTrnV1Lat = np.zeros((1750,(S1RoiMaskRep==sn).sum()))
predTstV1Lat = np.zeros((120,(S1RoiMaskRep==sn).sum()))
for i in range((S1RoiMaskRep==sn).sum()):
    
    predTrnV1Lat[:,i] = np.dot(np.delete(S1TrainDataRep[:,S1RoiMaskRep==sn],i,axis=1), 
                               df_wts_S1_avg.loc[src,src][i])
    predTstV1Lat[:,i] = np.dot(np.delete(S1TestDataRep[:,S1RoiMaskRep==sn],i,axis=1), 
                               df_wts_S1_avg.loc[src,src][i])
    
wts = np.concatenate(df_wts_S1_avg.loc[src][1:6])
predTrnV1 = np.dot(S1TrainDataRep[:,S1RoiMaskRep==sn], wts.T)
predTstV1 = np.dot(S1TestDataRep[:,S1RoiMaskRep==sn], wts.T)

trnAdj = origTrn - np.concatenate([predTrnV1Lat, predTrnV1],axis=1)
tstAdj = origTrn - np.concatenate([predTstV1Lat, predTstV1],axis=1)

In [111]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6], area_list[:6], trnAdj,tstAdj, lambda_list, 'average', .2, 
                                        roiNum, roiAdj)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S1_v1_adj_cc.pkl') 
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S1_v1_adj_wts.pkl')  
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S1_v1_adj_lambdas.pkl') 

In [6]:
### Subtract fwrf_pred from betas

fn = data_folder + '/results/fwRF/dnn_fwrf_Feb-17-2020_1345_repeated_trials_params.h5py'
valPred = h5py.File(fn)['val_pred'][:]
trnPred = h5py.File(fn)['trn_pred'][:]

## fwRF only has ROI voxels to cut down computatational time

trnAdj = S1TrainDataRep[:,S1RoiMaskRep>0]
tstAdj = S1TestDataRep[:,S1RoiMaskRep>0]
trnAdj = trnAdj - trn_pred
tstAdj = tstAdj - val_pred
roiAdj = S1RoiMaskRep[S1RoiMaskRep>0]

(1750, 8470) (120, 8470) (1750, 8470) (120, 8470) (8470,)


In [112]:
df_cc, df_wts, best_lambs = v2v.vox2vox(area_list[:6], area_list[:6], trnAdj,tstAdj, lambda_list, 'average', .2, 
                                        roiNum, roiAdj)

df_cc.to_pickle(data_folder + 'results/voxel2voxel/S1_fwrf_adj_cc.pkl') 
df_wts.to_pickle(data_folder + 'results/voxel2voxel/S1_fwrf_adj_wts.pkl')  
best_lambs.to_pickle(data_folder + 'results/voxel2voxel/S1_fwrf_adj_lambdas.pkl') 

### Mean Activity Analysis

In [185]:
df_cc = pd.DataFrame(np.nan, index=area_list[:6], columns=area_list[:6],dtype=object)

for s, src in enumerate(area_list[:6]):
    for t, trg in enumerate(area_list[:6]):
        
        if s!=t:
            
            df_cc.loc[src, trg] = v2v.corr2_coeff(np.nanmean(
                S1TrainDataRep[:,S1RoiMaskRep==roiNum[src]],axis=1)[:,np.newaxis],
                S1TrainDataRep[:,S1RoiMaskRep==roiNum[trg]])
            
        else:
            ccs = np.zeros((S1RoiMaskRep==roiNum[src]).sum())
            
            for v in range(len(ccs)):
                
                sv = np.delete(S1TrainDataRep[:,S1RoiMaskRep==roiNum[src]], v, axis=1)
                tv = S1TrainDataRep[:,S1RoiMaskRep==roiNum[src]][:,v]
                ccs[v]= v2v.sst.pearsonr(np.nanmean(sv, axis=1), tv)[0]
            df_cc.loc[src,trg]=ccs
        
df_cc.to_pickle(data_folder + 'results/voxel2voxel/S1_mean_avg_cc.pkl')         

In [ ]:
df_cc = pd.DataFrame(np.nan, index=area_list[:6], columns=area_list[:6],dtype=object)

for s, src in enumerate(area_list[:6]):
    for t, trg in enumerate(area_list[:6]):
        
        if s!=t:
            
            df_cc.loc[src, trg] = v2v.corr2_coeff(np.nanmean(
                S1TrainDataSing[:,S1RoiMaskSing==roiNum[src]],axis=1)[:,np.newaxis],
                S1TrainDataSing[:,S1RoiMaskSing==roiNum[trg]])
            
        else:
            ccs = np.zeros((S1RoiMaskSing==roiNum[src]).sum())
            
            for v in range(len(ccs)):
                
                sv = np.delete(S1TrainDataSing[:,S1RoiMaskSing==roiNum[src]], v, axis=1)
                tv = S1TrainDataSing[:,S1RoiMaskSing==roiNum[src]][:,v]
                ccs[v]= v2v.sst.pearsonr(np.nanmean(sv, axis=1), tv)[0]
            df_cc.loc[src,trg]=ccs
        
df_cc.to_pickle(data_folder + 'results/voxel2voxel/S1_mean_single_cc.pkl')         

In [187]:
df_cc = pd.DataFrame(np.nan, index=area_list[:6], columns=area_list[:6],dtype=object)

for s, src in enumerate(area_list[:6]):
    for t, trg in enumerate(area_list[:6]):
        
        if s!=t:
            
            df_cc.loc[src, trg] = v2v.corr2_coeff(np.nanmean(
                S2TrainDataRep[:,S2RoiMask==roiNum[src]],axis=1)[:,np.newaxis],
                S2TrainDataRep[:,S2RoiMask==roiNum[trg]])
            
        else:
            ccs = np.zeros((S2RoiMask==roiNum[src]).sum())
            
            for v in range(len(ccs)):
                
                sv = np.delete(S2TrainDataRep[:,S2RoiMask==roiNum[src]], v, axis=1)
                tv = S2TrainDataRep[:,S2RoiMask==roiNum[src]][:,v]
                ccs[v]= v2v.sst.pearsonr(np.nanmean(sv, axis=1), tv)[0]
            df_cc.loc[src,trg]=ccs
        
df_cc.to_pickle(data_folder + 'results/voxel2voxel/S2_mean_avg_cc.pkl')         

In [ ]:
df_cc = pd.DataFrame(np.nan, index=area_list[:6], columns=area_list[:6],dtype=object)

for s, src in enumerate(area_list[:6]):
    for t, trg in enumerate(area_list[:6]):
        
        if s!=t:
            
            df_cc.loc[src, trg] = v2v.corr2_coeff(np.nanmean(
                S2TrainDataSing[:,S2RoiMask==roiNum[src]],axis=1)[:,np.newaxis],
                S2TrainDataSing[:,S2RoiMask==roiNum[trg]])
            
        else:
            ccs = np.zeros((S2RoiMask==roiNum[src]).sum())
            
            for v in range(len(ccs)):
                
                sv = np.delete(S2TrainDataSing[:,S2RoiMask==roiNum[src]], v, axis=1)
                tv = S2TrainDataSing[:,S2RoiMask==roiNum[src]][:,v]
                ccs[v]= v2v.sst.pearsonr(np.nanmean(sv, axis=1), tv)[0]
            df_cc.loc[src,trg]=ccs
        
df_cc.to_pickle(data_folder + 'results/voxel2voxel/S2_mean_single_cc.pkl')         